In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)
design.chips.main.size['size_x']='3mm'
design.chips.main.size['size_y']='3mm'

In [18]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '2mm',
   'size_y': '2mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [22]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='main'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="50um",
                                              cpw_stub_height="200um",
                                              chip="main"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [23]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="main",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="main",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [24]:
stg=ShortToGround(design, 'stg', options=Dict(chip="main",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [25]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="main", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [26]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="main",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [30]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 # subtract coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.807562247348839]


In [39]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','800um']
jogs1[3]=['R','200um']
jogs1[4]=['R','600um']
jogs1[5]=['L','200um']
jogs1[6]=['L','600um']


In [40]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '918um', '100um')
            

# EPR Analysis

In [33]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [41]:
hfss.start()

INFO 10:57AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:57AM [load_ansys_project]: 	Opened Ansys App
INFO 10:57AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 10:57AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project1
INFO 10:57AM [connect_design]: No active design found (or error getting active design).
INFO 10:57AM [connect]: 	 Connected to project "Project1". No design detected


True

In [42]:
hfss.activate_ansys_design("Planar", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

10:57AM 31s WARNING [activate_ansys_design]: The design_name=Planar was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 10:57AM [connect_design]: 	Opened active design
	Design:    Planar [Solution type: Eigenmode]
WARNING 10:57AM [connect_setup]: 	No design setup detected.
WARNING 10:57AM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:57AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [43]:
hfss.close()


Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [44]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [45]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.81',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.81',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [46]:
c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 11:25AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:25AM [load_ansys_project]: 	Opened Ansys App
INFO 11:25AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:25AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS_Planar
INFO 11:25AM [connect_design]: 	Opened active design
	Design:    Planar [Solution type: Eigenmode]
INFO 11:25AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:25AM [connect]: 	Connected to project "KPS_Planar" and design "Planar" 😀 

INFO 11:25AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 11:25AM [connect_setup]: 	No design setup detected.
WARNING 11:25AM [connect_setup]: 	Creating Q3D default setup.
INFO 11:25AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:26AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:26AM [analyze]: Anal

,ground_main_plane,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground_main_plane,173.950,-47.0330,-41.052,-28.2980
pad_bot_Q0,-47.033,87.6010,-34.799,-2.1886
pad_top_Q0,-41.052,-34.7990,97.878,-18.9210
quad_coupler_c0,-28.298,-2.1886,-18.921,50.6100


In [47]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.05
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.684920 [GHz]
EC 303.451624 [MHz]
EJ 14.854145 [GHz]
alpha -349.182374 [MHz]
dispersion 14.326311 [KHz]
Lq 10.995564 [nH]
Cq 63.833002 [fF]
T1 39.911070 [us]

**Coupling Properties**

tCqbus1 -7.326965 [fF]
gbus1_in_MHz -114.992310 [MHz]
χ_bus1 -4.004796 [MHz]
1/T1bus1 3987.739365 [Hz]
T1bus1 39.911070 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.080523,350.304387,14.854145,-408.623621,53.667055,[-92.50661055392521],[-5.276432459595394],5.276432,92.506611
2,6.061697,347.98712,14.854145,-405.6498,50.598693,[-108.48851115140654],[-6.988555215253152],6.988555,108.488511
3,5.979645,337.990385,14.854145,-392.861478,38.974741,[-107.50592839531693],[-5.853410130985641],5.853410,107.505928
4,5.89351,327.675989,14.854145,-379.735856,29.398861,[-111.59969288512815],[-5.382200887548114],5.382201,111.599693
5,5.804315,317.187439,14.854145,-366.460076,21.758352,[-113.5602783134395],[-4.765371435400748],4.765371,113.560278
6,5.769389,313.133526,14.854145,-361.348091,19.289735,[-113.14482379529487],[-4.457860420385475],4.457860,113.144824
7,5.723242,307.822644,14.854145,-354.667222,16.414466,[-114.49992857252016],[-4.227244599755752],4.227245,114.499929
8,5.704363,305.66491,14.854145,-351.958106,15.353843,[-114.94520240873572],[-4.130167810692286],4.130168,114.945202
9,5.68492,303.451624,14.854145,-349.182374,14.326311,[-114.99231000954356],[-4.004795799008855],4.004796,114.992310


In [48]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
